# <center>1. Знакомимся с данными</center>

✍ Вы уже умеете делать запросы к одной таблице, использовать агрегатные функции и фильтровать данные в выводе. Но в реальных условиях базы данных обычно содержат множество таблиц и при запросе необходимо обращаться к нескольким таблицам. Освоением этого навыка мы сейчас и займёмся ↓

→ Первым делом — знакомимся с датасетом.

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме sql в [Metabase](http://sql.skillfactory.ru:3000/). Нам понадобятся таблицы teams и matches.

**Таблица teams с данными о командах**

|Название столбца|Содержимое столбца|
|---|---|
|id	|id команды|
|api_id	|ключ на таблицу matches|
|long_name	|полное название команды|
|short_name	|сокращённое название команды|

**Таблица matches с данными о матчах**

|Название столбца|Содержимое столбца|
|---|---|
|id	|id матча|
|season	|сезон|
|date	|дата матча|
|home_team_api_id	|api_id домашней команды, ключ на таблицу teams|
|away_team_api_id	|api_id гостевой команды, ключ на таблицу teams|
|home_team_goals	|количество голов домашней команды|
|away_team_goals	|количество голов гостевой команды|

Примечание. Как и в предыдущих модулях, вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.

Исследуйте данные датасета самостоятельно.  
→ Сколько различных полных названий команд в таблице teams?

```sql
SELECT COUNT(DISTINCT long_name) FROM sql.teams
```

→ Сколько в таблице teams команд с коротким названием VAL?

```sql
SELECT COUNT(short_name) FROM sql.teams WHERE short_name='VAL'
```

Информацию о скольких матчах содержит таблица matches?

```sql
SELECT COUNT(*) FROM sql.matches
```

→ Данные за какие сезоны даны в таблице matches?

```sql
SELECT season,COUNT(*) FROM sql.matches GROUP BY season ORDER BY season
```

Напишите запрос, который выведет сезон (season), а также общее количество забитых мячей домашними (total_home_goals) и гостевыми (total_away_goals) командами.  
Отсортируйте по столбцу с сезоном в порядке возрастания.

```sql
SELECT season, SUM(home_team_goals) AS total_home_goals, SUM(away_team_goals) AS total_away_goals
FROM sql.matches
GROUP BY season
ORDER BY season
```

# <center>2. Соединение таблиц по ключу</center>

## <center>Объединяем таблицы без операторов</center>

✍ Существует несколько способов соединения таблиц. Мы познакомимся со всеми основными операторами, которые используются для этих нужд, но начнём с простого метода объединения таблиц — без операторов.

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе from. Что произойдёт в таком случае?

```sql
SELECT * /*выбор всех полей*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
```

Каждая запись, которая есть в таблице teams, будет соединена с каждой записью в таблице matches.  
Это действие также называют декартовым произведением таблиц.

![](data/dst3-u2-md3_2_1.png)

Действительно ли это произведение?

Легко проверить! В исходных таблицах teams и matches было 299 и 25083 записей соответственно. Если соединить каждую запись одной таблицы с каждой записью другой, получится 299 * 25083 записей в итоговой таблице.

Напишите запрос, который выведет количество строк соединённой таблицы.

```sql
SELECT COUNT(*)
FROM sql.teams, sql.matches
```

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице teams есть столбец api_id, а таблица matches содержит столбцы home_team_api_id и away_team_api_id — это ключи таблиц, по которым они соединяются.

Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие
`where home_team_api_id = api_id`.

```sql
SELECT * /*выбор всех полей в таблице*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
`where away_team_api_id = api_id`.

```sql
SELECT * /*выбор всех полей в таблицы*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE away_team_api_id = api_id /*условие: away_team_api_id таблицы matches равен api_id таблицы teams*/
```

Итак, мы только что объединили таблицы по ключу.

Вы уже знакомы с ключами по таблице pokemon (там в этой роли выступал столбец id). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

→ Например, у нас есть несколько команд с одинаковым названием: Polonia Bytom, Widzew Łódź и Royal Excel Mouscron — хотя это разные команды, с разными id.
Кроме того, как мы уже смогли убедиться, ключи используются для соединения таблиц между собой.

Ключи бывают двух основных типов:

1. **Primary** — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.
2. **Foreign** — внешний ключ — представляет собой ссылку на другую таблицу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.  
Обратите внимание! В данном датасете ключ api_id таблицы teams может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (home) или гостевой (away) команде.  
Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.  
С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

```sql
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Напишите запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:

* названия гостевых команд (long_name),
* количество забитых мячей домашней команды (home_team_goals),
* количество забитых мячей гостевой команды (away_team_goals).

```sql
SELECT long_name, home_team_goals, away_team_goals
FROM sql.teams, sql.matches
WHERE away_team_api_id = api_id
```

# <center>3. Знакомимся с JOIN</center>